## Imports

In [30]:
% reset
import numpy as np
import os
import nltk
import re
import gensim
from sklearn.model_selection import train_test_split
from helpers import *
from cleaners import *
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from nltk.corpus import wordnet
import pathlib
from multiprocessing import Process,Lock,Value
from pathlib import Path
from gensim.models import Word2Vec,TfidfModel,Phrases
from gensim.corpora import Dictionary
from sklearn.svm import SVC

%load_ext autoreload
%autoreload 2

Once deleted, variables cannot be recovered. Proceed (y/[n])? y
The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


## Some helpers used to convert docs to vecs using tf weighted word2vec averaging

In [31]:
# an iterator used by gensim to train models without loading the whole corpus in memory
class MyDocs(object):
    def __init__(self, dirname):
        self.dirname = dirname
 
    def __iter__(self):
        pathlist = Path(self.dirname).glob('**/*')
        for path in pathlist:
            path_in_str = str(path)
            if os.path.isfile(path_in_str):
                f=open(path_in_str)
                yield f.read().split()

In [32]:
def word2vec(docs_root,model_name=None):
    # Training a word2vec model, the bigram transformer tells gensim to create vectors for bigram phrases such New York
    save = model_name!=None
    model_path = f'models/wv_{model_name}'
    model_exists = os.path.isfile(model_path)
    dir_exists = os.path.isdir('models')

    wv_model = None
    
    if model_exists:
        wv_model = Word2Vec.load(model_path)
    else:
        docs = MyDocs(docs_root)
        wv_model = Word2Vec(docs, size=300, window=5, min_count=100, workers=6)
    
    if save:
        if not dir_exists:
            pathlib.Path.mkdir('models')
        if not model_exists:
            wv_model.save(model_path)

    return wv_model
    

In [33]:
def tf_idf_model(doc_path,model_name=None):
    model_path = f'models/tf_{model_name}'
    dict_path = f'models/dict_{model_name}'
    model_exists = os.path.isfile(model_path)
    dict_exists = os.path.isfile(dict_path)
    dir_exists = os.path.isdir('models')
    save = model_name!=None
    
    docs = MyDocs(doc_path)
    bow_dict = Dictionary.load(dict_path) if dict_exists else Dictionary(docs)
    corpus = [bow_dict.doc2bow(doc) for doc in docs]
    tf_model = TfidfModel.load(model_path) if model_exists else TfidfModel(corpus)
    if save:
        if not dir_exists:
            pathlib.Path.mkdir('models')
        if not model_exists:
            tf_model.save(model_path)
        if not dict_exists:
            bow_dict.save(dict_path)
    return corpus,bow_dict,tf_model
        

In [34]:
def tf_doc2vec_factory(bow_dict,tf_model,word2vec):
    def tf_doc2vec(doc_bow,raw_doc):
        tf_dict = {e0:e1 for e0,e1 in tf_model[doc_bow]}
        doc_tf_idf = np.array([tf_dict[tok_id] for tok_id in (bow_dict.token2id[tok] for tok in raw_doc if tok in word2vec.wv.vocab)])
        vecs = np.array([word2vec.wv[word] for idx,word in enumerate(raw_doc) if word in word2vec.wv.vocab])
        doc_vec = np.sum(doc_tf_idf.reshape(-1,1)*vecs,axis=0)
        return doc_vec
    return tf_doc2vec


In [35]:
def tf_wv_samples(model_prefix,root_dir,wv=None):
    # root dir is assumed to have children /pos and /neg
    # word 2 vec model is created using all descendents of the parent of root_dir
    # returns X,y 
    
    pos_dir = f'{root_dir}/pos'
    neg_dir = f'{root_dir}/neg'
    
    # get the word2vec model
    if wv is None:
        wv = word2vec(root_dir.split('/')[0],model_prefix)
        
    # get tf-idf models
    pos_corpus, pos_dict, pos_tf_model = tf_idf_model(pos_dir, f'{model_prefix}_pos')
    neg_corpus, neg_dict, neg_tf_model = tf_idf_model(neg_dir, f'{model_prefix}_neg')
    # get tf-doc2vec factories
    pos_tf_doc2vec = tf_doc2vec_factory(pos_dict, pos_tf_model, wv)
    neg_tf_doc2vec = tf_doc2vec_factory(neg_dict, neg_tf_model, wv)
    
    pos_vecs = []
    for bows,doc in zip(pos_corpus,MyDocs(pos_dir)):
        pos_vecs.append(pos_tf_doc2vec(bows,doc))
    pos_vecs = np.array(pos_vecs)
    
    neg_vecs = []
    for bows,doc in zip(neg_corpus,MyDocs(neg_dir)):
        neg_vecs.append(neg_tf_doc2vec(bows,doc))
    neg_vecs = np.array(neg_vecs)
    
    X = np.concatenate((pos_vecs,neg_vecs),axis=0)
    y = np.asarray([1]*pos_vecs.shape[0] + [-1]*pos_vecs.shape[0])
    
    return X, y, wv


## cleaning the data without POS tagging

###  cleaning train and test data

In [36]:
basic_cleaned_root = speedy(basic_cleaner, ['aclImdb/train/pos','aclImdb/train/neg'],10,target_suffix='clean_nltkTok_2')

### creating training and validation set

In [38]:
X,y,wv_basic = tf_wv_samples(model_prefix='basic_nltk_2', root_dir=f'{basic_cleaned_root}/train')
X_train,X_val,y_train,y_val = train_test_split(X, y, test_size=0.33, random_state=42)

### validation of the classifiers

In [39]:
clf = SVC(C=1)
clf.fit(X_train, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [40]:
np.mean(clf.predict(X_val) == y_val)

0.72981818181818181

### Accuracy on the test data

In [37]:
basic_cleaned_test = speedy(basic_cleaner,['aclImdb/test/pos','aclImdb/test/neg'],10,target_suffix='clean_nltkTok_2')

In [41]:
X_test,y_test,wv_basic = tf_wv_samples(model_prefix='basic_nltk_2', root_dir=f'{basic_cleaned_test}/test',wv=wv_basic)


In [42]:
np.mean(clf.predict(X_test) == y_test)

0.73760000000000003

## POS tagged documents

In [27]:
pos_cleaned_root = speedy(pos_cleaner, ['aclImdb/train/pos','aclImdb/train/neg','aclImdb/train/unsup'],15,target_suffix='pos')

In [28]:
X,y,wv_pos = tf_wv_samples(model_prefix='pos', root_dir=f'{pos_cleaned_root}/train')
X_train,X_val,y_train,y_val = train_test_split(X, y, test_size=0.33, random_state=42)

In [67]:
clf = SVC(C=1)
clf.fit(X_train, y_train)

SVC(C=1, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False)

In [68]:
np.mean(clf.predict(X_val) == y_val)

0.78181818181818186

In [ ]:
basic_cleaned_test = speedy(basic_cleaner,['aclImdb/test/pos','aclImdb/test/neg'],10,target_suffix='clean_nltkTok_2')

## Google pre-trained word2vecs

In [10]:
model = gensim.models.KeyedVectors.load_word2vec_format('./GoogleNews-vectors-negative300.bin', binary=True)  
basic_cleaned_root = speedy(basic_cleaner, ['aclImdb/train/pos','aclImdb/train/neg','aclImdb/train/unsup'],15,target_suffix='clean')
X,y = tf_wv_samples(model_prefix='google', root_dir=f'{basic_cleaned_root}/train')
X_train,X_val,y_train,y_val = train_test_split(X, y, test_size=0.33, random_state=42)

/home/alyswidan/anaconda3/lib/python3.6/site-packages/gensim/models/phrases.py:494: UserWarning: For a faster implementation, use the gensim.models.phrases.Phraser class
  warnings.warn("For a faster implementation, use the gensim.models.phrases.Phraser class")


In [11]:
clf = SVC(C=1)
clf.fit(X_train, y_train)
np.mean(clf.predict(X_val) == y_val)

0.7781818181818182